<a href="https://colab.research.google.com/github/merrymasti015/CAP2022GRP09/blob/main/PCAMZC321_Group09_FireEventDetection_InceptionResNetV2_GG2_Gray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Architecture

#### Importing packages 

In [1]:
# Import necessary modules.

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from tensorflow.keras import datasets, models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ModelSaveFolder = "/content/gdrive/MyDrive/Fire_ImageDataSet/ModelSave"


Drive = "/content/gdrive/MyDrive/"
## Define root folder
RootFolder = Drive+"Fire_ImageDataSet/OutputFiles"

### Where to save models

ModelSaveFolder = Drive+"Fire_ImageDataSet/ModelSave"

print(ModelSaveFolder)

Mounted at /content/gdrive
/content/gdrive/MyDrive/Fire_ImageDataSet/ModelSave


# Preparing the data 

#### Calling test and train image directories 

These directories were prepared in the other notebook. 

In [3]:
base_dir = '/content/gdrive/MyDrive/Fire_ImageDataSet/OutputFiles/GreyBaseDataSet'

train_dir = os.path.join(base_dir, 'Train')
train_dir_fire = os.path.join(train_dir, 'Fire')
train_dir_nofire = os.path.join(train_dir, 'Neutral')

test_dir = os.path.join(base_dir, 'Test')
test_dir_fire = os.path.join(test_dir, 'Fire')
test_dir_nofire = os.path.join(test_dir, 'Neutral')

In [4]:
train_dir_nofire

'/content/gdrive/MyDrive/Fire_ImageDataSet/OutputFiles/GreyBaseDataSet/Train/Neutral'

In [5]:
list = os.listdir(train_dir_fire) # dir directory path
number_files = len(list)
print(number_files)

875


In [6]:
list = os.listdir(train_dir_nofire) # dir directory path
number_files = len(list)
print(number_files)

900


In [7]:
list = os.listdir(test_dir_fire) # dir directory path
number_files = len(list)
print(number_files)

97


In [8]:
list = os.listdir(test_dir_nofire) # dir directory path
number_files = len(list)
print(number_files)

90


#### Data generator & data augmentation 

For the large dataset it is not convenient to load all the data into memory. So we use image data generator to load the data from hard disc to memory in small batch. We do the same of the training and test set. 

Further, when initiating the image data generator we can do the data augmentation. This is the step to create more data from existing data by transforming the image. This artificially provides more data to train. Here we use rotation, translation, shear, zooming and horizontal flip for data augmentation. Other transformations like verticle flip is not suitable. We only do the data augmentation in the training set and not on the validation and test set. 

### https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

#### https://www.pluralsight.com/guides/image-classification-using-tensorflow - Good

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(150, 150), 
                                                   batch_size=32,
                                                   class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir, 
                                                   target_size=(150, 150), 
                                                   batch_size=32,
                                                   class_mode='binary')

Found 1775 images belonging to 2 classes.
Found 187 images belonging to 2 classes.


In [10]:
train_generator.samples 

1775

In [11]:
test_generator.samples 

187

In [12]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2             # pretrained CNN          
conv_base = InceptionResNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                 include_top=False,               # Do not include the ImageNet classifier at the top.
                 input_shape=(150, 150, 3))




#conv_base = (weights='imagenet',                  # Load weights pre-trained on ImageNet.
#                include_top=False,               # Do not include the ImageNet classifier at the top.
#                input_shape=(150, 150, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 3, 3, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 13824)             0         
                                                                 
 dense (Dense)               (None, 256)               3539200   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 57,876,193
Trainable params: 57,815,649
Non-trainable params: 60,544
_________________________________________________________________


In [14]:
import tensorflow as tf
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


# Training the model 

We pass the training data from the train_generator. We train for 30 epochs. We pass the validation data from the validation_generator.  We get validation accuracy above to 90% from this. 

In [ ]:
history = model.fit(train_generator, epochs=30, 
                    validation_data=test_generator)

Epoch 1/30


Saving the model for the future use. 

In [ ]:
model_name = 'InceptionResNetV2_Gray.h5'

In [ ]:
# Saving Model
model_name = 'InceptionResNetV2_Gray.h5'
model.save(ModelSaveFolder+'/'+model_name)

In [ ]:
from tensorflow import keras

In [ ]:


model = keras.models.load_model(ModelSaveFolder+'/'+model_name)

#### Visualization of the learning 

Training gives four sets of values in each eopch

- Training accuracy 
- Validation accuracy 
- Training loss 
- Validation loss 

In [ ]:
# Dictionary to extract the numbers 
hist_dict = history.history

# Training and validation accuracy 
training_acc = hist_dict['acc']
validation_acc = hist_dict['val_acc']

# Training and validation loss 
training_loss = hist_dict['loss']
validation_loss = hist_dict['val_loss']

# Number of epochs 
epoches = range(1, 1 + len(training_acc))

#### Function to make plot 

In [ ]:
def plot_func(entity):
    
    '''
    This function produces plot to compare the performance 
    between train set and validation set. 
    entity can be loss of accuracy. 
    '''
    
    plt.figure(figsize=(8, 5))
    plt.plot(epoches, eval('training_' + entity), 'r')
    plt.plot(epoches, eval('validation_' + entity), 'b')
    plt.legend(['Training ' + entity, 'Validation ' + entity])
    plt.xlabel('Epoches')
    plt.ylabel(entity)
    plt.show()

In [ ]:
plot_func('loss')

In [ ]:
plot_func('acc')

#### Getting the labels and predictions 

In [ ]:
len(test_generator)

#test_generator

In [ ]:
# taking first batch from the generator 
img, label = test_generator[5] 
print(label)
print(img)

In [ ]:
# taking first batch from the generator 
img, label = test_generator[0] 

# Predicting the images from the first batch 
pred = np.round(model.predict(img)).flatten()

In [ ]:
len(img)

In [ ]:
pred

In [ ]:
# Numeric to semantic labels 
label_dict = {1.0: 'No fire', 0.0: 'Fire'}

# Generating collage of plots 
fig = plt.figure(figsize=(10, 9))
plt.title('Classification by the model')
plt.axis('off')

for i, img_i in enumerate(img[:20]):
    ax = fig.add_subplot(4, 5, i+1)
    plt.axis('off')
    plt.title(label_dict[pred[i]], y=-0.2)
    ax.imshow(img_i)

#### Extracting misclassified images

In [ ]:
print(range(len(test_generator)))

In [ ]:
# Lists for missed fire images and missed non-fire images
msd_fire = []
msd_nofire = []

# Iterating through all the batches 
#for j in range(31):
for j in range(len(test_generator)):
    print(j)
    img, label = test_generator[j] 
    pred = np.round(model.predict(img)).flatten()
    bool_list = label == pred

    # bool_list is False when there is misclassification 
    for i, e in enumerate(bool_list):
        if e == False:
            
            # separating labels (fire and non-fire)
            if label[i] == 0:
                msd_fire.append(img[i])
            else:
                msd_nofire.append(img[i])

#### Confusion matrix

In [ ]:
# total number of sample in test set in each class 
n_class = 187

# number of misclassified fire and non-fire images 
nm_fire, nm_nofire = len(msd_fire), len(msd_nofire)

# confusion matrix (flattened)
conf_mat = [n_class-nm_fire, nm_fire, nm_nofire, n_class-nm_nofire]

# visualization of confusion matrix 
fig = plt.figure(figsize=(4, 4))
for i, j in enumerate(conf_mat):
    ax = fig.add_subplot(2, 2, i+1)
    ax.imshow([[j]], vmin=0, vmax=1000, cmap='copper_r')
    ax.text(-0.2, 0.1, j, c='r', fontsize=30)
    ax.axis('off')

# bringing blocks tighter 
fig.tight_layout()
fig.show()

In [ ]:


# total number of sample in test set in each class 
n_classFire    = 97
n_classNeutral = 90

# number of misclassified fire and non-fire images 
nm_fire, nm_nofire = len(msd_fire), len(msd_nofire)

# confusion matrix (flattened)
conf_mat = [n_classFire-nm_fire, nm_fire, nm_nofire, n_classNeutral-nm_nofire]

# confusion matrix (flattened)
#conf_mat = [n_class-nm_fire, nm_fire, nm_nofire, n_class-nm_nofire]

# visualization of confusion matrix 
fig = plt.figure(figsize=(4, 4))
for i, j in enumerate(conf_mat):
    ax = fig.add_subplot(2, 2, i+1)
    ax.imshow([[j]], vmin=0, vmax=1000, cmap='copper_r')
    ax.text(-0.2, 0.1, j, c='r', fontsize=30)
    ax.axis('off')

# bringing blocks tighter 
fig.tight_layout()
fig.show()

#### Showing mis-classified fire images 

In [ ]:
fig = plt.figure(figsize=(10, 7))
plt.title('Fire images classified as non-fire')
plt.axis('off')
for i, img_i in enumerate(msd_fire):        
    ax = fig.add_subplot(4, 5, i+1)
    ax.imshow(img_i)
    ax.axis('off')

Some of the misclassified figure have fire but that is too small. So even human observer is easy to confuse with them. Though some of the big explicit fire images are misclassified too. May be that is painting of fire but not the picture. Misclassified fire images are mostly bonfire, stove fire, fire tourch, kitchen fire etc. This is not big surprise because there were not enough fire sample in training set in that categories.  

#### Showing mis-classified non-fire images. 

In [ ]:
fig = plt.figure(figsize=(10, 4))
plt.title('Non-fire images classified as fire')
plt.axis('off')
for i, img_i in enumerate(msd_nofire):        
    ax = fig.add_subplot(2, 5, i+1)
    ax.imshow(img_i)
    ax.axis('off')

# Fine tuning the model

#### Unlocking the top convolutional block 

We trained previosuly with only top layer removed from VGG16. Here we unlock top base layer from VGG16 and fine tune the model. Doing so we reduce the learning rate from $10^{-4}$ to $10^{-5}$. We train for the 10 epoches. The model surpass the validation accuracy of 97% shortly after 30 epochs. It is not unlikely to improve the model after 50 epochs. But I am happy with this for now. The future plan is to check with other pre-trained model rather. 

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    else:
        set_trainable = False
    
model.compile(loss='binary_crossentropy', 
             optimizer=optimizers.RMSprop(lr=1e-5), 
             metrics=['acc'])

#### Fitting the model 

In [ ]:
history = model.fit_generator(train_generator, epochs=10, 
                             validation_data=test_generator)

In [ ]:
model_name = 'InceptionResNetV2_Prediction_fine_tuned_gray.h5'

model.save(ModelSaveFolder+'/'+model_name)



#### Visualization of fine tuning process

In [ ]:
hist_dict = history.history

training_accuracy = hist_dict['acc']
validation_accuracy = hist_dict['val_acc']

training_loss = hist_dict['loss']
validation_loss = hist_dict['val_loss']

epoches = range(1, 1 + len(training_accuracy))





In [ ]:
plot_func('loss')

In [ ]:
plot_func('accuracy')

# Error Analysis

In this section we analyze the error of the model, i.e. mis-classified images. We first see few examples of the correctly classified images. Then we visualize the confusion matrix. And finally, we see separately fire images classified as non-fire and non-fire images classified as fire.  

In [ ]:
from tensorflow import keras

In [ ]:
model_name = 'InceptionResNetV2_Prediction_fine_tuned_gray.h5'

In [ ]:
# Loading the saved model 
model = keras.models.load_model(ModelSaveFolder+'/'+model_name)


#### Getting the labels and predictions 

In [ ]:
len(test_generator)

#test_generator

In [ ]:
# taking first batch from the generator 
img, label = test_generator[5] 
print(label)
print(img)

In [ ]:
# taking first batch from the generator 
img, label = test_generator[0] 

# Predicting the images from the first batch 
pred = np.round(model.predict(img)).flatten()

In [ ]:
len(img)

In [ ]:
pred

In [ ]:
# Numeric to semantic labels 
label_dict = {1.0: 'No fire', 0.0: 'Fire'}

# Generating collage of plots 
fig = plt.figure(figsize=(10, 9))
plt.title('Classification by the model')
plt.axis('off')

for i, img_i in enumerate(img[:20]):
    ax = fig.add_subplot(4, 5, i+1)
    plt.axis('off')
    plt.title(label_dict[pred[i]], y=-0.2)
    ax.imshow(img_i)

#### Extracting misclassified images

In [ ]:
print(range(len(test_generator)))

In [ ]:
# Lists for missed fire images and missed non-fire images
msd_fire = []
msd_nofire = []

# Iterating through all the batches 
#for j in range(31):
for j in range(len(test_generator)):
    print(j)
    img, label = test_generator[j] 
    pred = np.round(model.predict(img)).flatten()
    bool_list = label == pred

    # bool_list is False when there is misclassification 
    for i, e in enumerate(bool_list):
        if e == False:
            
            # separating labels (fire and non-fire)
            if label[i] == 0:
                msd_fire.append(img[i])
            else:
                msd_nofire.append(img[i])

#### Confusion matrix

In [ ]:
# total number of sample in test set in each class 
n_class = 195

# number of misclassified fire and non-fire images 
nm_fire, nm_nofire = len(msd_fire), len(msd_nofire)

# confusion matrix (flattened)
conf_mat = [n_class-nm_fire, nm_fire, nm_nofire, n_class-nm_nofire]

# visualization of confusion matrix 
fig = plt.figure(figsize=(4, 4))
for i, j in enumerate(conf_mat):
    ax = fig.add_subplot(2, 2, i+1)
    ax.imshow([[j]], vmin=0, vmax=1000, cmap='copper_r')
    ax.text(-0.2, 0.1, j, c='r', fontsize=30)
    ax.axis('off')

# bringing blocks tighter 
fig.tight_layout()
fig.show()

In [ ]:
# total number of sample in test set in each class 
#n_class = 195

# number of misclassified fire and non-fire images 
#nm_fire, nm_nofire = len(msd_fire), len(msd_nofire)

# total number of sample in test set in each class 
n_classFire    = 97
n_classNeutral = 90

# number of misclassified fire and non-fire images 
nm_fire, nm_nofire = len(msd_fire), len(msd_nofire)

# confusion matrix (flattened)
conf_mat = [n_classFire-nm_fire, nm_fire, nm_nofire, n_classNeutral-nm_nofire]

# confusion matrix (flattened)
#conf_mat = [n_class-nm_fire, nm_fire, nm_nofire, n_class-nm_nofire]

# visualization of confusion matrix 
fig = plt.figure(figsize=(4, 4))
for i, j in enumerate(conf_mat):
    ax = fig.add_subplot(2, 2, i+1)
    ax.imshow([[j]], vmin=0, vmax=1000, cmap='copper_r')
    ax.text(-0.2, 0.1, j, c='r', fontsize=30)
    ax.axis('off')

# bringing blocks tighter 
fig.tight_layout()
fig.show()

#### Showing mis-classified fire images 

In [ ]:
fig = plt.figure(figsize=(10, 7))
plt.title('Fire images classified as non-fire')
plt.axis('off')
for i, img_i in enumerate(msd_fire):        
    ax = fig.add_subplot(4, 5, i+1)
    ax.imshow(img_i)
    ax.axis('off')

Some of the misclassified figure have fire but that is too small. So even human observer is easy to confuse with them. Though some of the big explicit fire images are misclassified too. May be that is painting of fire but not the picture. Misclassified fire images are mostly bonfire, stove fire, fire tourch, kitchen fire etc. This is not big surprise because there were not enough fire sample in training set in that categories.  

#### Showing mis-classified non-fire images. 

In [ ]:
fig = plt.figure(figsize=(10, 4))
plt.title('Non-fire images classified as fire')
plt.axis('off')
for i, img_i in enumerate(msd_nofire):        
    ax = fig.add_subplot(2, 5, i+1)
    ax.imshow(img_i)
    ax.axis('off')

Looking at this mis-classified set some of the picture actually seem to have fire. So, the problem is about the mis-labeling. Others don't have fire but have artificial red light or are picture with hue of dawn and dusk almost appearing as fire. 

Overall the model has done very good job separating those images with solid 97% accuracy in out of sample images. 

## Apply new set 

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

In [ ]:
# New set are kept C:/1-GG/CAP5/FireDetection/ApplySet/ApplyImageSet
Drive = "/content/gdrive/MyDrive/"
## Define root folder
RootFolder = Drive+"Fire_ImageDataSet/OutputFiles"
newImagePath = Drive + 'Fire_ImageDataSet/ApplySet'

ModelSaveFolder = "/content/gdrive/MyDrive/Fire_ImageDataSet/ModelSave"


In [ ]:
model_name = 'InceptionResNetV2_Prediction_fine_tuned_gray.h5'

In [ ]:
model_name

In [ ]:
# Loading the saved model 
model = keras.models.load_model(ModelSaveFolder+'/'+model_name)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generatorapply = test_datagen.flow_from_directory(newImagePath, 
                                                   target_size=(150, 150), 
                                                   batch_size=32,
                                                   class_mode='binary')

In [ ]:
len(test_generatorapply)

In [ ]:
test_generatorapply[len(test_generatorapply)-1]

In [ ]:
print(len(test_generatorapply))

#test_generator

In [ ]:
# taking first batch from the generator 
#img, label = test_generatorapply[0] 
img, label = test_generatorapply[len(test_generatorapply)-1] 
print(label)
print(img)

In [ ]:
# taking first batch from the generator 
img, label = test_generatorapply[0] 

# Predicting the images from the first batch 
pred = np.round(model.predict(img)).flatten()

In [ ]:
len(img)

In [ ]:
pred

In [ ]:
pred[3]

In [ ]:
# Numeric to semantic labels 
label_dict = {1.0: 'No fire', 0.0: 'Fire'}

# Generating collage of plots 
fig = plt.figure(figsize=(10, 9))
plt.title('Classification by the model')
plt.axis('off')

for i, img_i in enumerate(img[:20]):
    ax = fig.add_subplot(4, 5, i+1)
    plt.axis('off')
    plt.title(label_dict[pred[i]], y=-0.2)
    ax.imshow(img_i)

##  E N D 